In [43]:
import numpy as np
import loopy as lp
import pyopencl as cl
import pyopencl.array

platform = cl.get_platforms()[0]

# this is the GPU on my machine
device = platform.get_devices()[1]
ctx = cl.Context(devices=[device])
queue = cl.CommandQueue(ctx)

num_images = 64
num_tiles = 64
tw, th = (6, 6)
w, h = (128, 128)
images = cl.array.arange(queue, num_images*w*h, dtype=np.float32).reshape((num_images,w,h))
weights = cl.array.arange(queue, tw*th*num_tiles, dtype=np.float32).reshape((num_tiles, tw, th))

In [44]:
# Convolution over 3d (image, width, height)

knl = lp.make_kernel(
       "{ [x,y,tile,image,i,j]: 0<=x<w and 0<=y<h and 0<=tile<num_tiles and 0<=image<num_images and 0<=i,j<tile_size}",
       """
       out[image,tile,x,y] = input[image,x+i,y+j] * weights[tile,i,j]
       """,
       assumptions="w,h,num_tiles,tile_size,num_images>0 and tile_size mod 6 = 0 and w mod 16 = 0 and h mod 16 = 0")


knl = lp.set_loop_priority(knl, "image,tile,x,y,i,j")

knl = lp.split_iname(knl,  "x", 4, outer_tag="g.0", inner_tag="l.0")
knl = lp.split_iname(knl,  "y", 4, outer_tag="g.1", inner_tag="l.1")
knl = lp.split_iname(knl,  "tile", 4, outer_tag="g.2", inner_tag="l.2")
knl = lp.split_iname(knl,  "i", 4, inner_tag="unr")
knl = lp.split_iname(knl,  "j", 4, inner_tag="unr")

knl = lp.add_dtypes(knl, { "out": np.float32, "weights": np.float32, "input": np.float32 })
knl = lp.set_options(knl, 'write_cl')

evt, (out,) = knl(queue, input=images, weights=weights, tile_size=6, w=123, h=123)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(4, 4, 4))) loopy_kernel(int const h, __global float const *restrict input, int const num_images, int const num_tiles, __global float *restrict out, int const tile_size, int const w, __global float const *restrict weights)
{

  if (-1 + -4 * gid(2) + -1 * lid(2) + num_tiles >= 0)
    for (int image = 0; image <= -1 + num_images; ++image)
      for (int i_outer = 0; i_outer <= -1 + ((3 + tile_size) / 4); ++i_outer)
      {
        for (int j_outer = 0; j_outer <= -1 + ((3 + tile_size) / 4); ++j_outer)
        {
          out[h * w * num_tiles * image + h * w * (lid(2) + gid(2) * 4) + h * (lid(0) + gid(0) * 4) + lid(1) + gid(1) * 4] = input[(-1 + h + tile_size) * (-1 + w + tile_size) * image + (-1 + h + tile_size) * (lid(0) + gid(0) * 4 + 0 + i_outer * 4) + lid(1) + gid(1) * 4 + 0 + j_outer * 4] * weights[tile_size * tile_size * (lid(2) + gid(2) * 4) + tile_siz

In [64]:
# map (assume a flattened array)

knl = lp.make_kernel(
       "{ [x]: 0<=x<n }",
       """
       out[x] = tanh(input[x])
       """,
       assumptions="n>0")


knl = lp.set_loop_priority(knl, "x")
knl = lp.split_iname(knl,  "x", 8, outer_tag="g.0", inner_tag="unr", slabs=(0,1))
knl = lp.set_options(knl, 'write_cl')

in_array = cl.array.arange(queue, 1000, dtype=np.float32)
evt, out_array = knl(queue, input=in_array)

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *restrict input, int const n, __global float *restrict out)
{

  /* bulk slab for 'x_outer' */

  if (-9 + -8 * gid(0) + n >= 0)
  {
    out[0 + gid(0) * 8] = tanh(input[0 + gid(0) * 8]);
    out[1 + gid(0) * 8] = tanh(input[1 + gid(0) * 8]);
    out[2 + gid(0) * 8] = tanh(input[2 + gid(0) * 8]);
    out[3 + gid(0) * 8] = tanh(input[3 + gid(0) * 8]);
    out[4 + gid(0) * 8] = tanh(input[4 + gid(0) * 8]);
    out[5 + gid(0) * 8] = tanh(input[5 + gid(0) * 8]);
    out[6 + gid(0) * 8] = tanh(input[6 + gid(0) * 8]);
    out[7 + gid(0) * 8] = tanh(input[7 + gid(0) * 8]);
  }
  /* final slab for 'x_outer' */

  if (8 + 8 * gid(0) + -1 * n >= 0)
  {
    out[0 + gid(0) * 8] = tanh(input[0 + gid(0) * 8]);
    if (-2 + -8 * gid(0) + n >= 0)
      out[1 + gid(0) * 8] = tanh(input[1 + gid(0) * 8]);
    if (-3 + -8 * gid(0) + 

In [32]:
# statistics

from loopy.statistics import get_op_poly
import pprint
op_map = get_op_poly(knl)
op_map.dict[np.dtype(np.float32)].eval_with_dict({
        'h': w,
        'w': h,
        'num_tiles': num_tiles,
        'tile_size': tw,
        'num_images': num_images
    }) / 1e9

2.415919104